In [0]:
dbutils.widgets.dropdown(name = 'environment',choices=['dev', 'uat', 'prod'], defaultValue='dev', label='select the environment')
environment = dbutils.widgets.get('environment')    

In [0]:
bronze_url = spark.sql(f"DESCRIBE EXTERNAL LOCATION medallion_bronze_{environment}").select('url').collect()[0][0]

In [0]:
silver_url = spark.sql(f"DESCRIBE EXTERNAL LOCATION medallion_silver_{environment}").select('url').collect()[0][0]

In [0]:
gold_url = spark.sql(f"DESCRIBE EXTERNAL LOCATION medallion_gold_{environment}").select('url').collect()[0][0]

In [0]:
def create_schema(evironment, schema, path):
    spark.sql(f"""
              USE CATALOG '{evironment}_catalog'
              """)
    spark.sql(f"""
              DROP SCHEMA IF EXISTS {schema} CASCADE
              """)
    spark.sql(f"""
              CREATE SCHEMA IF NOT EXISTS {schema} MANAGED LOCATION '{path}'
              """)
    print(f'successfully create schema:{schema} in {evironment}_catalog, managed_location: {path}')

In [0]:
create_schema(environment, 'bronze', bronze_url)
create_schema(environment, 'silver', silver_url)
create_schema(environment, 'gold', gold_url)

In [0]:
def createTable_rawTraffic(environment):
    print(f'Creating raw_Traffic table in {environment}_catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `{environment}_catalog`.`bronze`.`raw_traffic`
                        (
                            Record_ID INT,
                            Count_point_id INT,
                            Direction_of_travel VARCHAR(255),
                            Year INT,
                            Count_date VARCHAR(255),
                            hour INT,
                            Region_id INT,
                            Region_name VARCHAR(255),
                            Local_authority_name VARCHAR(255),
                            Road_name VARCHAR(255),
                            Road_Category_ID INT,
                            Start_junction_road_name VARCHAR(255),
                            End_junction_road_name VARCHAR(255),
                            Latitude DOUBLE,
                            Longitude DOUBLE,
                            Link_length_km DOUBLE,
                            Pedal_cycles INT,
                            Two_wheeled_motor_vehicles INT,
                            Cars_and_taxis INT,
                            Buses_and_coaches INT,
                            LGV_Type INT,
                            HGV_Type INT,
                            EV_Car INT,
                            EV_Bike INT,
                            Extract_Time TIMESTAMP
                    );""")

In [0]:
def createTable_rawRoad(environment):
    print(f'Creating raw_roads table in {environment}_catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `{environment}_catalog`.`bronze`.`raw_roads`
                        (
                            Road_ID INT,
                            Road_Category_Id INT,
                            Road_Category VARCHAR(255),
                            Region_ID INT,
                            Region_Name VARCHAR(255),
                            Total_Link_Length_Km DOUBLE,
                            Total_Link_Length_Miles DOUBLE,
                            All_Motor_Vehicles DOUBLE
                    );""")

In [0]:
# environment = 'dev'
createTable_rawTraffic(environment)
createTable_rawRoad(environment)

In [0]:
# %sql
# DROP TABLE IF EXISTS dev_catalog.bronze.raw_roads;
# DROP TABLE IF EXISTS dev_catalog.bronze.raw_traffic;
# DROP TABLE IF EXISTS dev_catalog.silver.silver_roads;
# DROP TABLE IF EXISTS dev_catalog.silver.silver_traffic;
# DROP TABLE IF EXISTS dev_catalog.gold.gold_roads;
# DROP TABLE IF EXISTS dev_catalog.gold.gold_traffic;

In [0]:
# %sql
# DROP TABLE IF EXISTS dev_catalog.bronze.raw_roads;
# DROP TABLE IF EXISTS dev_catalog.bronze.raw_traffic;